In [27]:
import os

In [28]:
%pwd

'D:\\Documentation_sample'

In [29]:
os.chdir("D:\Documentation_sample")

In [30]:
%pwd

'D:\\Documentation_sample'

In [35]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is BEE5-8E65

 Directory of D:\Documentation_sample

25-07-2024  18:42    <DIR>          .
24-07-2024  10:42    <DIR>          .github
24-07-2024  17:50             3,314 .gitignore
24-07-2024  17:52    <DIR>          .vscode
24-07-2024  17:47    <DIR>          artifacts
24-07-2024  10:43    <DIR>          config
24-07-2024  11:02                 0 dvc.yaml
24-07-2024  12:37    <DIR>          logs
24-07-2024  17:46               443 main.py
24-07-2024  17:56        23,282,781 MNIST.zip
25-07-2024  18:04               139 params.yaml
24-07-2024  10:12                22 README.md
24-07-2024  16:17               134 requirements.txt
24-07-2024  17:59    <DIR>          research
24-07-2024  11:14               819 setup.py
25-07-2024  18:58    <DIR>          src
24-07-2024  10:53             1,308 template.py
24-07-2024  10:56    <DIR>          templates
25-07-2024  18:42    <DIR>          test  floder
24-07-2024  12:03                1

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file : Path
    unzip_dir: Path

In [43]:
from sample_project.constants import *
from sample_project.utils.common import read_yaml, create_directories

In [44]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [46]:
import os
import urllib.request as request
import zipfile
from sample_project import logger
from sample_project.utils.common import get_size

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 


    def extract_zip_file(self):
            """
            zip_file_path: str
            Extracts the zip file into the data directory
            Function returns None
            """
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path) 

In [48]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-25 19:22:49,363: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-25 19:22:49,363: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-25 19:22:49,367: INFO: common: created directory at: artifacts]
[2024-07-25 19:22:49,369: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-25 19:22:49,370: INFO: 2246197242: File already exists of size: ~ 22737 KB]
